In [18]:
import pandas as pd
import numpy as np

In [19]:
def get_data(url):
    data_frame = pd.read_csv(url)
    return data_frame

In [20]:
def remove_empty_value(data_frame):
    # Chuyển đổi số tập thành dữ liệu dạng số
    data_frame["episodes"] = pd.to_numeric(data_frame["episodes"], errors='coerce')
    # Xóa dòng dữ liệu khiếm khuyết
    data_frame = data_frame.dropna()

    return data_frame

In [21]:
def split_genre(genre_data):
    genre_data = genre_data.transform(lambda text: text.str.split(', '))
    return genre_data

In [22]:
def avg_rating(genre ,dataframe):
    rating = []
    for index, row in dataframe.iterrows():
        for item in row.genre:
            if item == genre:
                rating.append(row.rating)
                break
    return sum(rating)/len(rating)

In [23]:
# Tính giá trị trung bình rating cho từng thể loại => thể hiện sự ảnh hưởng của từng thể loại đối với rating.
def avg_genre_rating(dataframe):
    dict = {}
    for row in dataframe.genre:
        for item in row:
            if item in dict.keys():
                pass
            else:
                dict[item] = avg_rating(item, dataframe)
    return dict

In [24]:
# Tính giá trị trung bình rating của toàn bộ thể loại có trong một bộ anime.
def cal_weight_of_genre(items,dict):
    sum = 0
#     print(items)
    for item in items:
        sum += dict[item]
    return (sum/len(items))

In [25]:
def count_and_cal_weight_type(types):
    dict = {}
    for type in types:
        if type in dict.keys():
            dict[type] += 1
        else:
            dict[type] = 1
    return dict

In [26]:
# Thay genre thành trọng số
def replace_genre_to_numeric(genre_data,dict):
    genre_data = genre_data.transform(lambda items:cal_weight_of_genre(items,dict))
    return genre_data

In [27]:
# Thay type thành trọng số
def replace_type_to_numeric(type_data,dict):
    type_data = type_data.transform(lambda item:dict[item])
    return type_data

In [29]:
data_frame = get_data('/home/hyminhlee/Desktop/spark-3.0.1-bin-hadoop3.2/LR Final/archive2/archive/anime.csv')
data_frame = remove_empty_value(data_frame)

In [30]:
data_frame.genre = split_genre(data_frame.genre)

In [31]:
dict_genre = avg_genre_rating(data_frame)
dict_type = count_and_cal_weight_type(data_frame.type)

In [32]:
data_frame.genre  = replace_genre_to_numeric(data_frame.genre,dict_genre)
data_frame.type   = replace_type_to_numeric(data_frame.type,dict_type)

In [34]:
data_frame = data_frame.drop(['anime_id','name','members'],axis=1)

data_frame.to_csv('data_processed.csv',index=False)